In [1]:
from operator import itemgetter

In [12]:
def interleave(sc,submissions_list):
    
    merge_rdd = sc.emptyRDD()
    
    for s in submissions_list:
        ls_rdd = parse_submission_with_value(sc,s[0],s[1])
        merge_rdd = merge_rdd.union(ls_rdd)
    
    
    merge_rdd = (merge_rdd.flatMap(lambda r: [(r[0], x) for x in r[1]])
                 .map(lambda x: ((x[0],x[1][0]),x[1][1]))
                 .reduceByKey(lambda x,y:max(x,y))
                 .map(lambda x: (x[0][0],[(x[0][1],x[1])]))
                 .reduceByKey(lambda x,y:x+y)
                 .map(lambda x: (x[0],sorted(x[1], key=itemgetter(1), reverse=True)[:30]))
                 )
    
    return merge_rdd

def recommendation_list_with_value(string, value):
    updating_value = value
    lista=[]
    split = string.split(',')
    for i in split:
        if i != '':
            lista.append((int(i),updating_value))
            updating_value -= 1
            
    return lista

def parse_submission_with_value(sc, path, value):
    rdd = sc.textFile(path)
    
    header = rdd.first()

    rdd = (rdd.filter(lambda x: x != header)
    .map(lambda x: x.split("\t"))
    .filter(lambda x: x[0] != '')
    .map(lambda x:(int(x[0]),recommendation_list_with_value(x[1], value)))
    )

    return rdd

In [13]:
def recommendation_list(string):
    lista=[]
    split = string.split(',')
    for i in split:
        if i != '':
            lista.append((int(i)))
            
    return lista

def parse_submission(sc, path):
    rdd = sc.textFile(path)
    
    header = rdd.first()

    rdd = (rdd.filter(lambda x: x != header)
    .map(lambda x: x.split("\t"))
    .filter(lambda x: x[0] != '')
    .map(lambda x:(int(x[0]),recommendation_list(x[1])))
    )

    return rdd

In [4]:
a = parse_submission(sc, "/Users/tommy/Documents/pumpkin-pie/top_pop_impressions.csv")
b = parse_submission(sc, "/Users/tommy/Documents/pumpkin-pie/top_pop_impressions.csv")

print a.take(1)

[(9, [2778525, 1244196, 1053542, 657183, 536047, 1386412, 1162250, 1583705, 2791339, 784737, 1984327, 343377, 1576126, 2106311, 1056667, 830073, 1953846, 2446769, 1679143, 2031981, 1069281, 468120, 1851386, 1776330, 2593483, 2466095, 1443706, 2002097, 1092821, 460717])]


In [17]:
recommendation = interleave(sc, [("/Users/tommy/Documents/pumpkin-pie/top_pop_impressions.csv",30),
                                 ("/Users/tommy/Documents/pumpkin-pie/top_pop_impressions.csv",30)])
print recommendation.take(1)

[(458752, [(2778525, 30), (1244196, 29), (1053542, 28), (657183, 27), (536047, 26), (1386412, 25), (1162250, 24), (1583705, 23), (2791339, 22), (784737, 21), (1984327, 20), (343377, 19), (1576126, 18), (2106311, 17), (1056667, 16), (830073, 15), (1953846, 14), (2446769, 13), (1679143, 12), (2031981, 11), (1069281, 10), (468120, 9), (1851386, 8), (1776330, 7), (2593483, 6), (2466095, 5), (1443706, 4), (2002097, 3), (1092821, 2), (460717, 1)])]
